In [200]:
# !pip3 install python-rake
# !pip install summa
# !pip install pymorphy2

In [201]:
import os
import pandas as pd
import numpy as np
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from string import punctuation

import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize


morph = pymorphy2.MorphAnalyzer() 
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [202]:
def preprocess_text(text): #удаляет все лишние символы и лемматизирует, избавляется от регистра
    tokens = [token.strip('\\,.\'!?-:;\")(1234567890[]|%–“”={}×+⊃∈→«»—/‘’*').lower() \
              for token in text.split()]
    
    text = [morph.parse(token)[0].normal_form for token in tokens]
    
    tokens = ' '.join(token for token in text)
    return tokens

In [203]:
filelist = [] 
for root, dirs, files in os.walk('./texts'):
    for name in files:
        filelist.append(os.path.join(root, name))

corpus = []

for file in filelist:
    with open(file, 'r') as f:  
        text = f.read()
        corpus.append(text)

corpus_preprocessed =  [preprocess_text(text) for text in corpus]

##RAKE##

In [204]:
import RAKE

In [205]:
rake = RAKE.Rake(russian_stopwords)

In [206]:
rake_keywords = []
for text in corpus_preprocessed:
    rake_rez = rake.run(text, maxWords=3, minFrequency=1)
    kw = [rake_rez[i][0] for i in range(len(rake_rez))]
    rake_keywords.append(kw)

##TextRank##

In [207]:
from summa import keywords

In [208]:
textrank_keywords = []
for text in corpus_preprocessed:
    textrank_rez = keywords.keywords(text, language='russian', additional_stopwords=russian_stopwords, scores=True)
    tr_keywords = [textrank_rez[i][0] for i in range(len(textrank_rez))]
    textrank_keywords.append(tr_keywords)

##TFIDF##

Я взяла топ 50 слов для каждого текста с наибольшим значением tfidf, учитывались уни-, би- и триграммы

In [209]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [210]:
vectorizer = TfidfVectorizer(stop_words=russian_stopwords, ngram_range=(1,3))
X = vectorizer.fit_transform(corpus_preprocessed)
names = np.array(vectorizer.get_feature_names())

tfidf_keywords = [names[np.argsort(X.toarray()[i])[-50:]].tolist() for i in range(4)]

## Фильтры ##



Эталонные ключевые слова:
*русский язык; французский язык; одушевлённость; местоимения; анафорические употребления; дейктические употребления; *

*тональность текста; метод опорных векторов; наивный байесовский классификатор; random forest; классификация текстов; машинное обучение; социальные сети;*

*Ювентус; Зенит; Клаудиньо; Семак; Лига чемпионов;*

*Манчестер Юнайтед; Аталанта; Криштиану Роналду; Лига чемпионов.*

В целом, можно выделить следующие синтаксические  шаблоны:
- **Noun**
- **Adj + Noun**
- **Noun + Noun**
- **Adj + Adj + Noun**
- **Noun + Adj + Noun**

Морфологические характеристики я не добавляла, так все лемматизировала


Буду фильтровать выделенные ключевые слова по ним.

In [211]:
patterns = [['NOUN'], ['ADJF', 'NOUN'], ['NOUN', 'NOUN'], ['ADJF', 'ADJF', 'NOUN'], ['NOUN', 'ADJF', 'NOUN']]

def filter_kw(kw_list): 
    keywords_filtered = []
    for text_kw in kw_list:
        text_kw_filtered = []
        for kw in text_kw:
            if [morph.parse(word)[0].tag.POS for word in kw.split()] in patterns:
                text_kw_filtered.append(kw)
        keywords_filtered.append(text_kw_filtered)
    return keywords_filtered

rake_keywords_filtered = filter_kw(rake_keywords)
textrank_keywords_filtered = filter_kw(textrank_keywords)
tfidf_keywords_filtered = filter_kw(tfidf_keywords)


##Метрика##

In [212]:
keywords_init = [['Ювентус', 'Зенит', 'Клаудиньо', 'Семак', 'Лига чемпионов'], \
                 ['тональность текста', 'метод опорных векторов', 'наивный байесовский классификатор', 'random forest', 'классификация текстов', 'машинное обучение'],\
                 ['русский язык', 'французский язык', 'одушевлённость', 'местоимения', 'анафорические употребления', 'дейктические употребления'],\
                 ['Манчестер Юнайтед', 'Аталанта', 'Криштиану Роналду', 'Лига чемпионов']]

true_keywords = []
for keywords in keywords_init:
    true_keywords.append([preprocess_text(kw) for kw in keywords])
    

In [213]:
def count_precision(predicted_keywords, true_keywords):
    intersection = list(set(predicted_keywords) & set(true_keywords))
    return len(intersection)/len(predicted_keywords)


def count_recall(predicted_keywords, true_keywords):
    intersection = list(set(predicted_keywords) & set(true_keywords))
    return len(intersection)/len(true_keywords)

  
def count_f_measure(predicted_keywords, true_keywords):  #макро
    intersection = list(set(predicted_keywords) & set(true_keywords))
    precision = len(intersection)/len(predicted_keywords)
    recall = len(intersection)/len(true_keywords)
    if precision+recall > 0:
        f_measure = 2*(precision)*recall/(precision+recall)
    else:
        f_measure = 0
    return f_measure
    
rezults = []
for i in range(4):  #посчитаем отдельно для каждого текста
    precisions = [count_precision(kw[i], true_keywords[i]) for kw in [rake_keywords, textrank_keywords, tfidf_keywords, rake_keywords_filtered, textrank_keywords_filtered, tfidf_keywords_filtered]]
    recalls = [count_recall(kw[i], true_keywords[i]) for kw in [rake_keywords, textrank_keywords, tfidf_keywords, rake_keywords_filtered, textrank_keywords_filtered, tfidf_keywords_filtered]]
    f_measures = [count_f_measure(kw[i], true_keywords[i]) for kw in [rake_keywords, textrank_keywords, tfidf_keywords, rake_keywords_filtered, textrank_keywords_filtered, tfidf_keywords_filtered]]
    rez_df = pd.DataFrame([precisions, recalls, f_measures], columns=['RAKE', 'TextRank', 'TfIdf', 'RAKE_filtered', 'TextRank_filtered', 'TfIdf_filtered'], index=['Precision', 'Recall', 'F-measure'])
    rezults.append(rez_df)


In [214]:
def flatten(t):
    return [item for sublist in t for item in sublist]

#посчитаем метрики в общем
precisions = [count_precision(flatten(kw), flatten(true_keywords)) for kw in [rake_keywords, textrank_keywords, tfidf_keywords, rake_keywords_filtered, textrank_keywords_filtered, tfidf_keywords_filtered]]
recalls = [count_recall(flatten(kw), flatten(true_keywords)) for kw in [rake_keywords, textrank_keywords, tfidf_keywords, rake_keywords_filtered, textrank_keywords_filtered, tfidf_keywords_filtered]]
f_measures = [count_f_measure(flatten(kw), flatten(true_keywords)) for kw in [rake_keywords, textrank_keywords, tfidf_keywords, rake_keywords_filtered, textrank_keywords_filtered, tfidf_keywords_filtered]]
total_rezult = pd.DataFrame([precisions, recalls, f_measures], columns=['RAKE', 'TextRank', 'TfIdf', 'RAKE_filtered', 'TextRank_filtered', 'TfIdf_filtered'], index=['Precision', 'Recall', 'F-measure'])


## Результаты ##

**Текст 1 - футбольная экспертиза**: длина около 400 токенов

In [215]:
rezults[0]

,RAKE,TextRank,TfIdf,RAKE_filtered,TextRank_filtered,TfIdf_filtered
Precision,0.028986,0.058824,0.100000,0.111111,0.117647,0.192308
Recall,0.400000,0.400000,1.000000,0.400000,0.400000,1.000000
F-measure,0.054054,0.102564,0.181818,0.173913,0.181818,0.322581


Относительно хороший реколл, особенноо с использованием tfidf. Пресижн также лучший у tfidf, при этом фильтрация улучшает результаты (особенно для RAKE и TextRank). 

С этим текстом алгоритмы сработали лучше всего.

**Текст 2 - Статья про тональность**: длина около 1.5 тысяч токенов

In [216]:
rezults[1]

,RAKE,TextRank,TfIdf,RAKE_filtered,TextRank_filtered,TfIdf_filtered
Precision,0.021898,0.0,0.060000,0.035088,0.0,0.085714
Recall,0.500000,0.0,0.500000,0.333333,0.0,0.500000
F-measure,0.041958,0.0,0.107143,0.063492,0.0,0.146341


Снова наилучшие результаты получаются с методом tfidf, фильтрация также увеличивает метрики. Единственное, что в данном случае из-за фильтрации понизился реколл для RAKE, это связано с тем, что одно из ключевых слов для данного текста - random forest, морфпарсер определяет его часть речи как UNKN, а такой шаблон я не стала добавлять. Поэтому в фильтрованный RAKE он не вошел, хотя до фильтров алгоритм выделил это ключевое слово.

В целом, особо заметно что TextRank в данном случае ничего не нашел. Также и у других методов показатели значительно ниже в сравнении с предыдущим текстом. Возможно, это связано с некоторой сложностью ключевых слов, в данном случае были биграммы и триграммы. 

**Текст 3 - Статья про одушевленность и местоимения**: длина около 3 тысяч токенов

In [217]:
rezults[2]

,RAKE,TextRank,TfIdf,RAKE_filtered,TextRank_filtered,TfIdf_filtered
Precision,0.014052,0.006173,0.080000,0.044118,0.018519,0.153846
Recall,1.000000,0.166667,0.666667,1.000000,0.166667,0.666667
F-measure,0.027714,0.011905,0.142857,0.084507,0.033333,0.250000


В данном случае RAKE выделил все ключевые слова из эталона, однако из-за очень низкого пресижна лучшим методом снова стал tfidf, фильтрация опять же улучшила результат. Самые худшие показатели снова у TextRank. В целом, неплохой результат относительно других текстов

**Текст 4 - короткая новостная статья**: длина около 70 токенов

In [218]:
rezults[3]

,RAKE,TextRank,TfIdf,RAKE_filtered,TextRank_filtered,TfIdf_filtered
Precision,0.0,0.142857,0.040000,0.0,0.200000,0.090909
Recall,0.0,0.250000,0.500000,0.0,0.250000,0.500000
F-measure,0.0,0.181818,0.074074,0.0,0.222222,0.153846


В этом же случае лучше всех сработал TextRank с фильтрами, RAKE же вообще не обнаружил верных ключевых слов. Больший рекол вышел у tfidf, однако у для TextRank значительно выше пресижн.

**Общие результаты**:

In [219]:
total_rezult

,RAKE,TextRank,TfIdf,RAKE_filtered,TextRank_filtered,TfIdf_filtered
Precision,0.017134,0.016667,0.065000,0.046083,0.040323,0.119266
Recall,0.523810,0.238095,0.619048,0.476190,0.238095,0.619048
F-measure,0.033183,0.031153,0.117647,0.084034,0.068966,0.200000


**В целом, из таких результатов видно, что TextRank хорошо находит ключевые слова на коротких текстах и плохо на длинных (но это может быть просто совпадением), RAKE в целом справляется неплохо, но не отличительно, а самый стабильный и почти всегда лучший метод это TfIdf. Применение фильтров улучшает качество во всех случаях**

## Улучшения/изменения ##

 - Первое, что может (должно) улучшить качество, это установка меньшей длины ключевых слов. Я везде устанавливала (где могла) возможную длину последовательности от 1 до 3, но только для одном тексте в эталон ключевых слов были включены триграммы. Уменьшив длину, мы, вероятно, увеличили бы пресижн для 3 из 4 текстов, так как алгоритмы не стали бы выделять триграммы.

  Также во многих случаях некоторые биграммы, которые сами по себе являлись ключевами словами, алгоритмы включали в состав триграмм, что ухудшало качество. Например, rake выделяет 'лига чемпион доказывать', а ключевым является 'лига чемпион'. Однако если выделять только уни- и биграммы, то он может выделить верное 'лига чемпион', то есть правильных ответов будет больше.
 - Второе - оптимально подобрать количество выделяемых слов для tfidf. Я брала топ50, но это было скорее рандомное решение. Так может увеличиться пресижн
 - еще одним возможным улучшением может быть особый препроцессинг, а в частности стемминг. Я совсем в этом не уверена, но почему-то, мне кажется, это может помочь увеличить качество. Например, в статье про одушевленность одним из эталонных ключевых слов является 'одушевленность', а многие алгоритмы выделяют 'одушевленный'. При стемминге не было бы таких ошибок (хотя не всегда и ошибок).